<a href="https://colab.research.google.com/github/qwasd34/DEV_DATA/blob/main/%5B%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A7%88%EC%9D%B4%EB%8B%9D%5D_%E1%84%90%E1%85%A9%E1%84%91%E1%85%B5%E1%86%A8_%E1%84%86%E1%85%A9%E1%84%83%E1%85%A6%E1%86%AF%E1%84%85%E1%85%B5%E1%86%BC_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Yelp 데이터셋에 잠재하는 주제를 찾아내기

  입력: 전체 텍스트 문서 집합 / 토픽의수

  출력:
    문서별 토픽 분포 / 토픽별 단어분포

- 단어 레벨로 토큰 설정
- 기본적인 텍스트 데이터 전처리
  - 토크나이즈 (띄어쓰기)
  - Stop words 제거
  - Stemming
    - 많이 사용하는 PorterStemmer 사용
  - 기타
    - 소문자화(정규화) , 비단어적 요소 제거

# 데이터 로드



In [2]:
# txt 파일을 pandas의 형태로 변환하지 않고
# 직접적으로 load
# 과정에서 0 혹은 1 부분은 제거

file_path = 'yelp_labelled.txt'

with open(file_path, 'r', encoding='utf-8') as file :
    data = [line.split('\t')[0] for line in file]

print(data[:5])

['Wow... Loved this place.', 'Crust is not good.', 'Not tasty and the texture was just nasty.', 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.', 'The selection on the menu was great and so were the prices.']


# 전처리 (Tokenize, Stop Words, Stemming + alpha)

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocessing(text) :
    # 소문자 변환
    text = text.lower()
    # 비 단어적 요소 제거
    text = re.sub(r'\W', ' ', text)
    # Tokeinize (띄어쓰기 단위로)
    text = text.split()
    # Stop words 제거
    text = [t for t in text if t not in stop_words]
    # 어간 추출
    text = [stemmer.stem(word) for word in text]
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
for d in data[:5] :
    print(f'원래 문장 : {d}')
    print(f'전처리 후 문장 : {preprocessing(d)}')
    print()


원래 문장 : Wow... Loved this place.
전처리 후 문장 : ['wow', 'love', 'place']

원래 문장 : Crust is not good.
전처리 후 문장 : ['crust', 'good']

원래 문장 : Not tasty and the texture was just nasty.
전처리 후 문장 : ['tasti', 'textur', 'nasti']

원래 문장 : Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.
전처리 후 문장 : ['stop', 'late', 'may', 'bank', 'holiday', 'rick', 'steve', 'recommend', 'love']

원래 문장 : The selection on the menu was great and so were the prices.
전처리 후 문장 : ['select', 'menu', 'great', 'price']



In [5]:
preproc_data = [preprocessing(d) for d in data]
preproc_data[:5]

[['wow', 'love', 'place'],
 ['crust', 'good'],
 ['tasti', 'textur', 'nasti'],
 ['stop',
  'late',
  'may',
  'bank',
  'holiday',
  'rick',
  'steve',
  'recommend',
  'love'],
 ['select', 'menu', 'great', 'price']]

# LDA 모델 적용하기



## Document-Term Matrix 생성

- 문서 단어 행렬 (DTM)

- 전처리된 데이터에서 각단어의 빈도를 나타내는 행렬 생성
- 행의 방향으로 문서를 나타내며, 열의 방향으로 단어를 나타냄
- 어떤 열번호 index에 어떤 단어 term 가 들어갈지 정해야함
  - Gensim 패키지의 Dictionary Class 이용
- 만들어진 단어 term - 번호index 객체를 활용 DTM 생성
  - (단어 ID, 빈도수) 의 형태

In [6]:
from gensim import corpora

# Gensim의 Dictionary 객체를 생성
dictionary = corpora.Dictionary(preproc_data)

In [7]:
list(dictionary.token2id.items())[:10]

[('love', 0),
 ('place', 1),
 ('wow', 2),
 ('crust', 3),
 ('good', 4),
 ('nasti', 5),
 ('tasti', 6),
 ('textur', 7),
 ('bank', 8),
 ('holiday', 9)]

In [8]:
# 문서-단어 행렬을 생성
corpus = [dictionary.doc2bow(text) for text in preproc_data]

In [10]:
# 첫 번째 문장의 DTM
# 0번째 단어가 1번 1번째 단어가 1번 ...!
corpus[0]

[(0, 1), (1, 1), (2, 1)]

In [11]:
# 이는 0번째 단어가 1번, 1번째 단어가 1번, 2번째 단어가 1번 나옴을 의미
# 각 단어를 확인하기 위해서는 아래의 과정을 통해 알 수 있음

print(f'0 번째 단어 : {dictionary[0]}')
print(f'1 번째 단어 : {dictionary[1]}')
print(f'2 번째 단어 : {dictionary[2]}')

0 번째 단어 : love
1 번째 단어 : place
2 번째 단어 : wow


## LDA 적용

In [13]:
from gensim.models import ldamodel

topicK = 3
num_trains = 10

lda_model = ldamodel.LdaModel(corpus,
                              num_topics=topicK, # 선정한 토픽 수
                              id2word=dictionary,# 몇번째 단어가 무엇인지
                              passes=num_trains, # 학습 횟수
                              random_state=42)

In [14]:
# 토픽 별 단어 분포 확인 (토픽, 그토픽을 구성하는데 얼만큼 쓰였는지)
for k in range(topicK):
    print(lda_model.show_topic(k, topn=20))

[('servic', 0.017086321), ('good', 0.014856899), ('disappoint', 0.010963259), ('like', 0.009364962), ('best', 0.009119847), ('great', 0.008761751), ('also', 0.00869444), ('realli', 0.008605385), ('place', 0.008238193), ('staff', 0.007839445), ('friendli', 0.007232641), ('order', 0.006750442), ('eat', 0.0066319024), ('nice', 0.0062933825), ('could', 0.0058348477), ('steak', 0.0052713323), ('amaz', 0.005116638), ('even', 0.0050651086), ('know', 0.004823529), ('burger', 0.004761508)]
[('food', 0.042987805), ('back', 0.02138683), ('good', 0.019900084), ('servic', 0.017604794), ('place', 0.017388444), ('go', 0.017267667), ('time', 0.011744485), ('wait', 0.010304522), ('would', 0.008815033), ('ever', 0.0072248215), ('love', 0.0067911996), ('minut', 0.006650361), ('never', 0.006449497), ('come', 0.0062793614), ('pizza', 0.0056791217), ('like', 0.0056220596), ('vega', 0.005453013), ('want', 0.005266863), ('great', 0.0050176657), ('recommend', 0.0048496644)]
[('place', 0.021426972), ('great', 0

In [15]:
# 문서 별 토픽 분포 확인
for document in corpus[:5]:
    origin_doc = [dictionary[word_idx] for word_idx, word_num in document]
    print(f'{origin_doc}에 속한 토픽의 분포는 아래와 같습니다.')
    for topic_idx, topic_dist in lda_model[document]:
        print(f'{topic_idx} 번째 토픽 : {topic_dist*100:.2f}% 확률')

['love', 'place', 'wow']에 속한 토픽의 분포는 아래와 같습니다.
0 번째 토픽 : 8.79% 확률
1 번째 토픽 : 10.01% 확률
2 번째 토픽 : 81.20% 확률
['crust', 'good']에 속한 토픽의 분포는 아래와 같습니다.
0 번째 토픽 : 75.95% 확률
1 번째 토픽 : 12.74% 확률
2 번째 토픽 : 11.31% 확률
['nasti', 'tasti', 'textur']에 속한 토픽의 분포는 아래와 같습니다.
0 번째 토픽 : 11.01% 확률
1 번째 토픽 : 80.56% 확률
2 번째 토픽 : 8.43% 확률
['love', 'bank', 'holiday', 'late', 'may', 'recommend', 'rick', 'steve', 'stop']에 속한 토픽의 분포는 아래와 같습니다.
0 번째 토픽 : 3.47% 확률
1 번째 토픽 : 93.11% 확률
2 번째 토픽 : 3.43% 확률
['great', 'menu', 'price', 'select']에 속한 토픽의 분포는 아래와 같습니다.
0 번째 토픽 : 35.37% 확률
1 번째 토픽 : 7.65% 확률
2 번째 토픽 : 56.98% 확률


In [16]:
# 토픽 별 단어 분포 확인
for k in range(topicK):
    print(f'{k}번째 토픽을 구성하는 단어의 분포는...')
    for word, prob in lda_model.show_topic(k, topn=5) :
        print(f'{word} : {prob*100:.2f}%', end= ' ')
    print()
    print()

0번째 토픽을 구성하는 단어의 분포는...
servic : 1.71% good : 1.49% disappoint : 1.10% like : 0.94% best : 0.91% 

1번째 토픽을 구성하는 단어의 분포는...
food : 4.30% back : 2.14% good : 1.99% servic : 1.76% place : 1.74% 

2번째 토픽을 구성하는 단어의 분포는...
place : 2.14% great : 1.77% restaur : 0.78% like : 0.73% salad : 0.72% 



# LDA 모델 결과 해석하기

- 토픽의 의미는 사용자가 선정해야함
  - 토픽을 구정하는 단어를 보고 토픽의 의미를 선정


In [17]:
# 0번째 토픽을 구성하는 상위 10개 단어
for word, prob in lda_model.show_topic(0, topn=10):
    print(f'{word}, {prob*100:.2f}%')

servic, 1.71%
good, 1.49%
disappoint, 1.10%
like, 0.94%
best, 0.91%
great, 0.88%
also, 0.87%
realli, 0.86%
place, 0.82%
staff, 0.78%


- 식당 서비스 품질과 고객 경험
  - service best place staff

In [18]:
# 1번째 토픽을 구성하는 상위 10개 단어
for word, prob in lda_model.show_topic(1, topn=10):
    print(f'{word}, {prob*100:.2f}%')

food, 4.30%
back, 2.14%
good, 1.99%
servic, 1.76%
place, 1.74%
go, 1.73%
time, 1.17%
wait, 1.03%
would, 0.88%
ever, 0.72%


- 음식 품질과 재방문에 관련된 의사표현
  -food back, good

In [19]:
# 2번째 토픽을 구성하는 상위 10개 단어
for word, prob in lda_model.show_topic(2, topn=10):
    print(f'{word}, {prob*100:.2f}%')

place, 2.14%
great, 1.77%
restaur, 0.78%
like, 0.73%
salad, 0.72%
star, 0.70%
delici, 0.65%
time, 0.64%
get, 0.54%
pretti, 0.54%


- 식당 전반의 분위기와 음식의 퀄리티
  - place great restaurnt delicious

In [21]:
# 원문장과 토픽 분포를 보고 해석하기
#랜덤한 인덱스
target_idx = 10

print('원 문장 : ', data[target_idx])
print('전처리 문장 : ', preproc_data[target_idx])
print('문장 내 토픽 분포 : ')
for topic_idx, prob in lda_model[corpus[target_idx]]:
    print(f'  - {topic_idx}번 토픽 : {prob*100:.2f}%')

원 문장 :  Service was very prompt.
전처리 문장 :  ['servic', 'prompt']
문장 내 토픽 분포 : 
  - 0번 토픽 : 76.57%
  - 1번 토픽 : 12.26%
  - 2번 토픽 : 11.16%
